In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
from pyspark.sql.types import *
#import rdd
import pyspark.sql.functions as F
spark = SparkSession.builder.appName("MyFirstCSV").getOrCreate()


In [ ]:
#CREATE DATAFRAME FROM CSV FILE
oij_df = spark.read.csv( path="OIJ.csv", sep=",", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

In [ ]:
#Function to remove spaces from the beginning and end of the string
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, trim(col))
    return df

#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

oij_df.show(5)
inec_df.show(5)

In [ ]:
#Function find list the not match values in the dataframes
def find_non_matches(df1,df2):
    non_matches = []
    for col in df1.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            for row in df1.select(col).distinct().collect():
                if not df2.filter(df2['Provincia, cantón y distrito'] == row[col]).collect():
                    non_matches.append(row[col])
    return non_matches

non_matches_oij = find_non_matches(oij_df,inec_df)


#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)
#oij_df.show(5)
#inec_df.show(5)


In [ ]:
#Function to replace the accents in column Provincia, cantón y distrito in inec_df
def replace_accents(df):
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'á', 'a'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'é', 'e'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'í', 'i'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ó', 'o'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ú', 'u'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ñ', 'n'))
    return df

inec_df = replace_accents(inec_df)
nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)


In [ ]:
#Fucntion to separate column Provincia, cantón y distrito in inec_df in three columns


In [ ]:
def generate_new_columns(df):
    columns = ['Provincia', 'Canton', 'Distrito','Tasa neta de participación', 'Porcentaje de población económicamente inactiva', 'Relación de dependencia económica']
    dfFromRDD2 = spark.createDataFrame(rdd).toDF(*columns)



    first_column =  df.columns[0]
    
    counter = 0
    for row in first_column:
        if row == '':
            counter += 1
        if counter == 1:
            new_df.withColumn('Provincia', row)
        if counter == 2:
            new_df.withColumn('Canton', row)
        if counter == 3:
            new_df.withColumn('Distrito', row)
            counter = 0
    return new_df
        


In [ ]:
#Function to remove column Provincia, cantón y distrito in inec_df
def remove_column(df):
    df = df.drop('Provincia, cantón y distrito')
    return df

inec_df = remove_column(inec_df)

In [ ]:
#Function to create a new dataframe with inec_df and oij_df when provincia, canton and distrito are equals
def join_dataframes(df1,df2):
    df = df1.join(df2, (df1.Provincia == df2.ProvinciaUnida) | (df1.Canton == df2.CantonUnido) | (df1.Distrito == df2.DistritoUnido), 'inner')
    return df

df = join_dataframes(oij_df,inec_df)
df.show(25)


In [ ]:
#Create sql table from dataframe
df.createOrReplaceTempView("df")

#Save the dataframe in a postgresql database
df.write.jdbc(url="jdbc:postgresql://localhost:5432/etl", table="etl", mode="overwrite", properties={"user": "postgres", "password": "Legolas00"})


In [ ]:
#Save the dataframe in csv file
df.write.csv(path="output.csv", sep=",", header=True,quote='"',mode="overwrite")
df.show(5)
